In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import yaml
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import Image, display
from typing import List

In [ ]:
# Experiments to visualize
exp_names = [
    "demo",
]

In [ ]:
from utils.eval_utils import EditResultData

# Load all results
eval_dirs = sum([sorted([p for p in Path(f"result/{exp_name}").glob("*/") if p.is_dir()]) for exp_name in exp_names], [])
edit_results = [EditResultData.from_metrics(eval_dir, skip_img_load=True) for eval_dir in eval_dirs]

In [ ]:
def get_metrics(edit_results: List[EditResultData]) -> pd.DataFrame:
    """Creates a pandas Dataframe for all experiments

    Args:
        edit_results (List[EditResultData]): Experiment results

    Returns:
        pd.DataFrame Results as pandas dataframe
    """

    data = []

    for r in tqdm(edit_results):
        method_cfg = {**r.method}

        metrics = {
            "data": str(r.data),
            "exp_name": r.exp_name,
            "method": method_cfg.pop("type"),
            "scheduler": method_cfg.pop("scheduler"),
            "steps": method_cfg.pop("num_inference_steps", 50),
            "edit_method": r.edit_method["type"],
            "method_cfg": method_cfg,
            **{k: v["mean"] for k, v in r.metrics.items()},
        }

        data.append(metrics)

    print(data)

    return pd.DataFrame(data)

df = get_metrics(edit_results)
df

In [ ]:
def get_metrics_per_image_single(exp: EditResultData) -> pd.DataFrame:
    """Get per image metrics for one experiment

    Args:
        exp (EditResultData): Result object to retrieve per image metrics for

    Returns:
        pd.DataFrame: Dataframe
    """

    out = []
    for i in range(len(exp)):
        sample = exp[i]
        out.append({
            **{k: sample[k] for k in ("image_file", "edit_image_file", "source_prompt", "target_prompt")},
            **sample["metrics"],
            "method": exp.method,
            "edit_method": exp.edit_method,
        })

    if len(out) == 0:
        return None

    out = pd.DataFrame(out)

    return out

def get_metrics_per_image(edit_results: List[EditResultData]) -> pd.DataFrame:
    """Get per image metrics for multiple experiment

    Args:
        edit_results (List[EditResultData]): Result objects to retrieve per image metrics for

    Returns:
        pd.DataFrame: Dataframes
    """
    
    out = [get_metrics_per_image_single(exp) for exp in edit_results]
    out = pd.concat([o for o in out if o is not None])

    return out

df_per_img = get_metrics_per_image(edit_results[2:3])
df_per_img

In [ ]:
from utils.vis_utils import img_grid

def show_grid(entrys: pd.DataFrame, topk: int=5) -> "DisplayHandle":
    """Display a image grid of per image results.

    Args:
        entrys (pd.DataFrame): Per image dataframe
        topk (int, optional): How many entries to visualize. Defaults to 5.

    Returns:
        DisplayHandle: Image for notebook display
    """

    rows = []
    rows_titles = []

    def create_row(entry):
        rows.append([str(entry.image_file), str(entry.edit_image_file)])
        rows_titles.append([entry.source_prompt, "\n".join([entry.target_prompt, entry.method["type"], entry.edit_method["type"]])])

    for i in range(topk):
        create_row(entrys.iloc[i])

    out = img_grid(rows, rows_titles)
    return display(Image(cv2.imencode(".png", out)[1]))

show_grid(df_per_img, 3)